In [18]:
import torch
import torchvision
import torchvision.transforms as transforms

Load device

In [20]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
#device = "cuda" if torch.cuda.is_available()  else "cpu"
torch.device(device)


device(type='mps')

Load Data

In [21]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])

train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)

test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


Data loaders

In [22]:
num_epochs = 5
batch_size = 40
learning_rate = 0.001

train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)

n_total_step = len(train_loader)
print(n_total_step)

1250


Creare instance of pre-trained VGG16 model

In [23]:
from torchvision import models
import torch.nn as nn

model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)

/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Look the network

In [25]:
from torchinfo import summary

summary(model, input_size=(batch_size, 3, 224, 224), device=device, col_names=['input_size', 'output_size',
                                                                              'num_params'])

/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/Users/manuelrodriguez/miniconda3/envs/pytorch2/lib/python3.8/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [40, 3, 224, 224]         [40, 10]                  --
├─Sequential: 1-1                        [40, 3, 224, 224]         [40, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [40, 3, 224, 224]         [40, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [40, 64, 224, 224]        [40, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [40, 64, 224, 224]        [40, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [40, 64, 224, 224]        [40, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [40, 64, 224, 224]        [40, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [40, 64, 112, 112]        [40, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [40, 128, 112, 112]       [40, 128, 112, 112]       --
│    └─Conv2d: 2-8      

set up cost and optimizer

In [26]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

Train the model


In [28]:
for epoch in range(1):
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 250 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%')
            print()

KeyboardInterrupt: 